# MLP

In [ ]:
target_col = 'value'


target_mean = dfs[target_col].mean()
target_stdev = dfs[target_col].std()

for c in dfs.columns:
    mean = dfs[c].mean()
    stdev = dfs[c].std()

    dfs[c] = (dfs[c] - mean) / stdev

    
from sklearn.model_selection import train_test_split

y = dfs[[target_col]]
X = dfs.drop(columns=[target_col])

from sklearn.linear_model import LinearRegression


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0805, shuffle=False)

In [ ]:
model = MLPRegressor(random_state=1, activation='identity',  learning_rate='constant', learning_rate_init=0.0003,
                     early_stopping=True,  validation_fraction=0.0917, verbose=False, power_t=0.5, batch_size=32, alpha=0.0005, shuffle=False,
                      max_iter=60, solver='adam',  hidden_layer_sizes=(128,2)).fit(X_train, y_train)


In [ ]:
prediction = model.predict(X_test)
result = pd.DataFrame(y_test)
result["Model forecast"] = prediction
result = result.sort_index()


for c in result.columns:
    result[c] = result[c] * target_stdev + target_mean


df_test_plot = result.reset_index(drop=False)

df_out = df_test_plot[df_test_plot['date']>='2020-10-01 01:00:00']



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
def calculate_metrics(df_test_plot):
    return {'mae' : mean_absolute_error(df_test_plot.value, df_test_plot['Model forecast']),
            'rmse' : mean_squared_error(df_test_plot.value, df_test_plot['Model forecast']) ** 0.5,
            'r2' : r2_score(df_test_plot.value, df_test_plot['Model forecast']),
             'mape' : mean_absolute_percentage_error(df_test_plot.value, df_test_plot['Model forecast'])*100}


result_metrics = calculate_metrics(df_test_plot)

# Prophet

In [ ]:
from prophet import Prophet

model = Prophet(weekly_seasonality=False, daily_seasonality=False, seasonality_mode='additive')

In [ ]:
ex_features = ['lag24', 'zap_lag24', 'zrodla_wiatrowe_lag24', "is_weekend", 'lag48', 'lag72', 'lag96', 'lag120', 'lag144','lag168', 'lag336', 'zap_lag48',
             'zrodla_wiatrowe_lag48', 'zap_lag72', 'zrodla_wiatrowe_lag72', 'zap_lag96', 'zrodla_wiatrowe_lag96',
              'zap_lag120', 'zrodla_wiatrowe_lag120', 'zap_lag144', 'zrodla_wiatrowe_lag144', 'zap_lag168', 'zrodla_wiatrowe_lag168',
              'zap_lag336', 'zrodla_wiatrowe_lag336', 'kod_dnia']

In [ ]:
for feature in ex_features:
    model.add_regressor(feature)
    
model.fit(df_train[["date", "value"] + ex_features].rename(columns={"date": "ds", "value": "y"}))

forecast = model.predict(df_test[["date", "value"] + ex_features].rename(columns={"date": "ds"}))

forecast_ = forecast.loc[:, forecast.columns.intersection(['ds','yhat'])]
forecast_ = forecast_.rename(columns={"ds": "date"})
df_test_ =  df_test.loc[:, df_test.columns.intersection(['date','value'])]
new_df = pd.merge(df_test_, forecast_, how='left', on=['date'])

new_df = new_df.set_index("date")

for c in new_df.columns:
    new_df[c] = new_df[c] * target_stdev + target_mean
    
df_test_plot = new_df

df_test_plot = df_test_plot.reset_index(drop=False)
df_test_plot = df_test_plot.rename(columns={"yhat": "Model forecast"})
